IMPORTATION DES MODULES

In [66]:
import requests #pip install requests
from xml.etree import ElementTree #pip install xml.etree # Pour lire les données extraites par l'api arXiv
import datetime as dt #pip install datetime #Pour convertir les dates de publicaiton de str en datetime pour les ranger par ordre chronologique

# 3 modules pour l'extraction du texte du pdf, manque tjrs les images...
import io
import pdfminer #pip install pdfminer
from pdfminer.high_level import extract_text  #pip install pdfminer.six

Création de l'API, présentation

In [67]:
from flask import Flask, Response


app = Flask(__name__)

@app.route('/')
def index():
    return 'Présentation de notre API'



Appel à l'API et stockage des données dans une liste pour éviter de faire plein de fois le même appel pour la même requête.

In [68]:
def callAPI(urlreq, param):
    arxiv_url = urlreq
    research_type = param
    rep = requests.get(arxiv_url, params=research_type)
    root = ElementTree.fromstring(rep.content)
    return root

def articlesAPI():
    articles = []
    # Ici on défini notre url de requête et nos paramètres pour faire appel à l'api
    # On va chercher partout où le titre contient AI, et on garde 5 artciles comme dit la consigne
    root = callAPI("http://export.arxiv.org/api/query?",{"search_query": "all:ai", "max_results" : 5})
    # On regarde élément par élément dans notre recherche : le titre/l'abstract/le nom de l'auteur/
    #la date de publication/le lien pour accéder à l"article/l'identifiant associé
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        title = entry.find('{http://www.w3.org/2005/Atom}title').text
        abs = entry.find('{http://www.w3.org/2005/Atom}summary').text
        auteur = entry.find('{http://www.w3.org/2005/Atom}author').find('{http://www.w3.org/2005/Atom}name').text
        published = entry.find('{http://www.w3.org/2005/Atom}published').text[0:10]
        link = entry.find('{http://www.w3.org/2005/Atom}id').text
        id = link[21:]
        urlpdf = f"http://arxiv.org/pdf/{id}"


        articles.append([id, link, auteur, title, published, urlpdf, abs])

    for article in articles: # pour trier les artciles dans l'ordre chronologik ; pas trouvé comment faire avec l'api directement
        article[4] = dt.datetime.strptime(article[4], '%Y-%m-%d')
    articles.sort(key=lambda x: x[4], reverse = True)
    return articles

larticles = articlesAPI()

1ère route : get_data

récupère 5 liens d'articles et les range par ordre chronologique de sortie, puis les affiche

AMODIF, un peu nul

In [69]:
@app.route('/get_data')
def get_data():
    liste = larticles
    a = [i[1] for i in liste]
    html = "<br>".join(a)
    return Response(html, mimetype='text/html')

get_data()



['http://arxiv.org/abs/2402.07632v1', 'http://arxiv.org/abs/2403.05551v1', 'http://arxiv.org/abs/2305.15922v1', 'http://arxiv.org/abs/2211.05075v1', 'http://arxiv.org/abs/2103.15294v1']


<Response 181 bytes [200 OK]>

2ème route : Affiche les informations relatives aux 5 articles précédents, id, lien, auteur, titre, date de publi, lien pdf, abstract

In [70]:
@app.route('/articles')
def articles():
    art = larticles 
    commentaire = "Pour voir le contenu d'un article en particulier : modifier le chemin, /articles devient article/ID où ID est l'identifiant de l'article choisi"
    html = commentaire +  "<br>"+ "<br>".join([f"ID: {a[0]}<br>Link: {a[1]}<br>Authors: {a[2]}<br>Title: {a[3]}<br>Published: {a[4]}<br>PDF: {a[5]}<br>Summary: {a[6]}<br><br>" for a in art])
    return Response(html, mimetype='text/html')



<Response 5849 bytes [200 OK]>

3ème route : Affiche le lien pdf et son contenu d'un article identifié par son id

In [71]:
@app.route('/article/<id>')
def article_content(id):
    urlpdf = f"http://arxiv.org/pdf/{id}"
    response = requests.get(urlpdf)

    text = extract_text(io.BytesIO(response.content))

    # Convertir les sauts de ligne en balises <br> pour l'affichage HTML
    html_content = "Lien pour accéder au pdf direct : " + urlpdf +'<br>'+'<br>'+'<br>' + text.replace('\n', '<br>')

    return Response(html_content, mimetype='text/html')


In [72]:
@app.route('/ml') # reste à compléter
def ml_method():
    return 

In [73]:
# faire tourner le serveur

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Apr/2024 18:52:46] "GET /articles HTTP/1.1" 200 -
